In [1]:
pip install deepspeech ffmpeg-python pydub pysrt

ERROR: Could not find a version that satisfies the requirement deepspeech (from versions: none)
ERROR: No matching distribution found for deepspeech


In [ ]:
import os
import subprocess
import deepspeech
from pydub import AudioSegment
import pysrt

def extract_audio_from_video(video_path, audio_path):
    # Using ffmpeg to extract audio from video
    cmd = f"ffmpeg -i {video_path} -vn -acodec pcm_s16le -ar 16000 -ac 1 {audio_path}"
    subprocess.call(cmd, shell=True)

def transcribe_audio(model_path, audio_path):
    # Load DeepSpeech model
    model = deepspeech.Model(model_path)

    # Read the audio file
    audio = AudioSegment.from_wav(audio_path)

    # Split audio into smaller chunks for faster processing (optional)
    audio_chunks = [audio[start:start + 5000] for start in range(0, len(audio), 5000)]

    # Initialize an empty list to store transcriptions
    transcriptions = []

    # Perform speech-to-text on each chunk
    for chunk in audio_chunks:
        chunk_wav_path = "temp_chunk.wav"
        chunk.export(chunk_wav_path, format="wav")
        text = model.stt(chunk_wav_path)
        transcriptions.append(text)
        os.remove(chunk_wav_path)

    # Combine transcriptions into a single string
    transcription_text = " ".join(transcriptions)

    return transcription_text

def create_srt_file(transcription, srt_output_path):
    # Create a new SRT file
    subs = pysrt.SubRipFile()

    # Split the transcription into chunks of 10 seconds
    chunk_duration = 10000  # milliseconds (10 seconds)
    for idx, start in enumerate(range(0, len(transcription), chunk_duration)):
        end = start + chunk_duration
        subtitle = pysrt.SubRipItem(index=idx,
                                    start=pysrt.SubRipTime(milliseconds=start),
                                    end=pysrt.SubRipTime(milliseconds=end),
                                    text=transcription[start:end])
        subs.append(subtitle)

    # Save the SRT file
    subs.save(srt_output_path, encoding="utf-8")

if __name__ == "__main__":
    video_path = "path/to/your/video.mp4"
    audio_path = "temp_audio.wav"
    model_path = "path/to/your/deepspeech_model.pbmm"
    srt_output_path = "output_subtitle.srt"

    # Step 1: Extract audio from video
    extract_audio_from_video(video_path, audio_path)

    # Step 2: Transcribe the audio using DeepSpeech
    transcription = transcribe_audio(model_path, audio_path)

    # Step 3: Create SRT file from transcription
    create_srt_file(transcription, srt_output_path)

    print("Subtitles generated and saved to:", srt_output_path)
